In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive

drive.mount('/content/drive')

base_path = "/content/drive/MyDrive"

processed_path = os.path.join(base_path, "coil-20-proc")
unprocessed_path = os.path.join(base_path, "coil-20-unproc")

output_processed = os.path.join(base_path, "output_processed")
output_unprocessed = os.path.join(base_path, "output_unprocessed")

os.makedirs(output_processed, exist_ok=True)
os.makedirs(output_unprocessed, exist_ok=True)

def get_image_files(folder):
    return sorted([
        f for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and
        f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))
    ])

def segmentation_pipeline(image_path, visualize=False):

    img = cv2.imread(image_path)

    if img is None:
        print("Error loading:", image_path)
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray, (5,5), 0)

    sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobelx, sobely)
    sobel = np.uint8(np.absolute(sobel))

    canny = cv2.Canny(blur, 50, 150)

    thresh = cv2.adaptiveThreshold(
        blur,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        11,
        2
    )

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)

    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(gray)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest_contour], -1, 255, -1)

        x, y, w, h = cv2.boundingRect(largest_contour)
        boxed = img.copy()
        cv2.rectangle(boxed, (x,y), (x+w, y+h), (0,255,0), 2)
    else:
        boxed = img.copy()

    segmented = cv2.bitwise_and(img, img, mask=mask)

    if visualize:
        plt.figure(figsize=(16,10))

        plt.subplot(2,4,1)
        plt.imshow(gray, cmap='gray')
        plt.title("Grayscale")

        plt.subplot(2,4,2)
        plt.imshow(blur, cmap='gray')
        plt.title("Gaussian Blur")

        plt.subplot(2,4,3)
        plt.imshow(sobel, cmap='gray')
        plt.title("Sobel Edge")

        plt.subplot(2,4,4)
        plt.imshow(canny, cmap='gray')
        plt.title("Canny Edge")

        plt.subplot(2,4,5)
        plt.imshow(thresh, cmap='gray')
        plt.title("Adaptive Threshold")

        plt.subplot(2,4,6)
        plt.imshow(morph, cmap='gray')
        plt.title("After Morphology")

        plt.subplot(2,4,7)
        plt.imshow(mask, cmap='gray')
        plt.title("Object Mask")

        plt.subplot(2,4,8)
        plt.imshow(cv2.cvtColor(boxed, cv2.COLOR_BGR2RGB))
        plt.title("Bounding Box")

        plt.tight_layout()
        plt.show()

    return segmented

processed_images = get_image_files(processed_path)
unprocessed_images = get_image_files(unprocessed_path)

if processed_images:
    sample_processed = os.path.join(processed_path, processed_images[0])
    print("Showing pipeline for one PROCESSED image")
    segmentation_pipeline(sample_processed, visualize=True)
else:
    print("No processed images found!")

if unprocessed_images:
    sample_unprocessed = os.path.join(unprocessed_path, unprocessed_images[0])
    print("Showing pipeline for one UNPROCESSED image")
    segmentation_pipeline(sample_unprocessed, visualize=True)
else:
    print("No unprocessed images found!")

print("Processing all PROCESSED images...")
for img_name in processed_images:
    img_path = os.path.join(processed_path, img_name)
    result = segmentation_pipeline(img_path)
    if result is not None:
        cv2.imwrite(os.path.join(output_processed, img_name), result)

print("Processing all UNPROCESSED images...")
for img_name in unprocessed_images:
    img_path = os.path.join(unprocessed_path, img_name)
    result = segmentation_pipeline(img_path)
    if result is not None:
        cv2.imwrite(os.path.join(output_unprocessed, img_name), result)

print("Segmentation completed for entire dataset.")